In [44]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', None)

# Data Preparation

## Combine All Data Kata

In [5]:
def merge_csv_files_with_filename(directory):
    df_list = []

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            
            df = pd.read_csv(file_path)
            
            df['label'] = str(os.path.splitext(filename)[0]).capitalize() 
                
            df_list.append(df)

    combined_df = pd.concat(df_list, ignore_index=True)    
    return combined_df

directory = 'datasets/extraction-feature-default/Non-Merged'
combined_df = merge_csv_files_with_filename(directory)

combined_df.to_csv('datasets/extraction-feature-default/40-kata.csv', index=False)

## Labelling Alphabet

In [8]:
alphabet_df = pd.read_csv('./datasets/extraction-feature-default/alphabet.csv')
alphabet_df

,filename,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,...,z_12,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,A_0.jpg,0.155182,0.729192,2.719984e-07,0.205288,0.735770,-0.017337,0.275075,0.718441,-0.035706,0.329495,0.709703,-0.055288,0.374989,0.702892,-0.076279,0.293241,0.639296,-0.046337,0.369748,0.612025,-0.084543,0.422023,0.597921,-0.107750,...,-0.119020,0.214076,0.620907,-0.074296,0.272353,0.658624,-0.120460,0.250701,0.692648,-0.112077,0.228995,0.689233,-0.097932,0.166513,0.630084,-0.089888,0.217268,0.657916,-0.120492,0.212754,0.687127,-0.110377,0.198819,0.687841,-0.097879
1,A_1.jpg,0.838807,0.719874,-2.164804e-07,0.771837,0.725273,-0.009544,0.705295,0.701828,-0.023455,0.662583,0.677778,-0.039253,0.628528,0.655313,-0.056163,0.697544,0.624706,-0.041077,0.612177,0.596289,-0.075613,0.560344,0.577005,-0.094528,...,-0.091366,0.778687,0.613676,-0.073650,0.714158,0.669719,-0.111475,0.741234,0.699804,-0.090671,0.767731,0.690158,-0.068801,0.825814,0.631450,-0.092105,0.772004,0.671297,-0.121461,0.778908,0.694842,-0.108882,0.794608,0.690194,-0.093175
2,A_10.jpg,0.581184,0.745845,-1.059189e-07,0.557238,0.751812,-0.004449,0.536100,0.740906,-0.010580,0.519795,0.737985,-0.017672,0.502753,0.742129,-0.025208,0.539090,0.672356,-0.008672,0.520708,0.655394,-0.017746,0.509604,0.638421,-0.024760,...,-0.023058,0.561821,0.671356,-0.017525,0.537792,0.710742,-0.025926,0.544868,0.730442,-0.019770,0.553987,0.724883,-0.015368,0.572253,0.688464,-0.023423,0.551785,0.715056,-0.030928,0.554675,0.728463,-0.027442,0.562592,0.725510,-0.023784
3,A_100.jpg,0.374870,0.759894,4.366655e-08,0.398134,0.758703,-0.007355,0.421602,0.743577,-0.017515,0.442338,0.741927,-0.028168,0.462645,0.746148,-0.039076,0.413038,0.671357,-0.023561,0.437463,0.646744,-0.040337,0.453789,0.627753,-0.050616,...,-0.047230,0.383540,0.671320,-0.034483,0.411728,0.706482,-0.052234,0.408348,0.740305,-0.046356,0.400300,0.747717,-0.039503,0.370315,0.691556,-0.041090,0.393848,0.708879,-0.052244,0.397180,0.733071,-0.047251,0.392820,0.741922,-0.041883
4,A_102.jpg,0.143311,0.835336,1.923219e-07,0.214950,0.844010,-0.012342,0.296637,0.814950,-0.027893,0.355519,0.792106,-0.046894,0.400802,0.763687,-0.066535,0.302602,0.724712,-0.037958,0.395459,0.693706,-0.078667,0.456062,0.675371,-0.102087,...,-0.119402,0.215821,0.706110,-0.072580,0.282703,0.770142,-0.123735,0.246608,0.804842,-0.112590,0.217682,0.791075,-0.096030,0.170938,0.723427,-0.091741,0.227759,0.771976,-0.123583,0.207771,0.800647,-0.111719,0.182577,0.788167,-0.097839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7778,Z_95.jpg,0.188713,0.452745,4.376307e-08,0.186131,0.458152,-0.019916,0.175022,0.449016,-0.028224,0.159213,0.433381,-0.033037,0.144067,0.422552,-0.036739,0.161157,0.402638,-0.021101,0.135917,0.401613,-0.030524,0.118950,0.400954,-0.037094,...,-0.034458,0.151392,0.400473,-0.010523,0.129117,0.396233,-0.019902,0.113192,0.391546,-0.025209,0.100124,0.386310,-0.027908,0.148680,0.405670,-0.007579,0.131076,0.400261,-0.015862,0.118617,0.396493,-0.020378,0.108216,0.392156,-0.022700
7779,Z_96.jpg,0.189172,0.450795,4.833628e-08,0.186669,0.458279,-0.020497,0.175500,0.449759,-0.029007,0.159451,0.433658,-0.033871,0.144074,0.422893,-0.037607,0.161484,0.403710,-0.022008,0.135898,0.402940,-0.031654,0.118940,0.401783,-0.038212,...,-0.034948,0.152145,0.398767,-0.010865,0.129569,0.395513,-0.020315,0.113492,0.391658,-0.025511,0.100302,0.386949,-0.028114,0.149512,0.403153,-0.007699,0.131526,0.399041,-0.016022,0.119002,0.395974,-0.020522,0.108578,0.392031,-0.022807
7780,Z_97.jpg,0.188380,0.451641,4.892988e-08,0.186817,0.458061,-0.020542,0.176152,0.450207,-0.029136,0.160090,0.435203,-0.034156,0.144393,0.424882,-0.038083,0.161438,0.404872,-0.021769,0.136609,0.403409,-0.031627,0.11

In [13]:
alphabet_df['label'] = alphabet_df['filename'].apply(lambda x: x[0])
alphabet_df.head()

,filename,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,...,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,label
0,A_0.jpg,0.155182,0.729192,2.719984e-07,0.205288,0.735770,-0.017337,0.275075,0.718441,-0.035706,0.329495,0.709703,-0.055288,0.374989,0.702892,-0.076279,0.293241,0.639296,-0.046337,0.369748,0.612025,-0.084543,0.422023,0.597921,-0.107750,...,0.214076,0.620907,-0.074296,0.272353,0.658624,-0.120460,0.250701,0.692648,-0.112077,0.228995,0.689233,-0.097932,0.166513,0.630084,-0.089888,0.217268,0.657916,-0.120492,0.212754,0.687127,-0.110377,0.198819,0.687841,-0.097879,A
1,A_1.jpg,0.838807,0.719874,-2.164804e-07,0.771837,0.725273,-0.009544,0.705295,0.701828,-0.023455,0.662583,0.677778,-0.039253,0.628528,0.655313,-0.056163,0.697544,0.624706,-0.041077,0.612177,0.596289,-0.075613,0.560344,0.577005,-0.094528,...,0.778687,0.613676,-0.073650,0.714158,0.669719,-0.111475,0.741234,0.699804,-0.090671,0.767731,0.690158,-0.068801,0.825814,0.631450,-0.092105,0.772004,0.671297,-0.121461,0.778908,0.694842,-0.108882,0.794608,0.690194,-0.093175,A
2,A_10.jpg,0.581184,0.745845,-1.059189e-07,0.557238,0.751812,-0.004449,0.536100,0.740906,-0.010580,0.519795,0.737985,-0.017672,0.502753,0.742129,-0.025208,0.539090,0.672356,-0.008672,0.520708,0.655394,-0.017746,0.509604,0.638421,-0.024760,...,0.561821,0.671356,-0.017525,0.537792,0.710742,-0.025926,0.544868,0.730442,-0.019770,0.553987,0.724883,-0.015368,0.572253,0.688464,-0.023423,0.551785,0.715056,-0.030928,0.554675,0.728463,-0.027442,0.562592,0.725510,-0.023784,A
3,A_100.jpg,0.374870,0.759894,4.366655e-08,0.398134,0.758703,-0.007355,0.421602,0.743577,-0.017515,0.442338,0.741927,-0.028168,0.462645,0.746148,-0.039076,0.413038,0.671357,-0.023561,0.437463,0.646744,-0.040337,0.453789,0.627753,-0.050616,...,0.383540,0.671320,-0.034483,0.411728,0.706482,-0.052234,0.408348,0.740305,-0.046356,0.400300,0.747717,-0.039503,0.370315,0.691556,-0.041090,0.393848,0.708879,-0.052244,0.397180,0.733071,-0.047251,0.392820,0.741922,-0.041883,A
4,A_102.jpg,0.143311,0.835336,1.923219e-07,0.214950,0.844010,-0.012342,0.296637,0.814950,-0.027893,0.355519,0.792106,-0.046894,0.400802,0.763687,-0.066535,0.302602,0.724712,-0.037958,0.395459,0.693706,-0.078667,0.456062,0.675371,-0.102087,...,0.215821,0.706110,-0.072580,0.282703,0.770142,-0.123735,0.246608,0.804842,-0.112590,0.217682,0.791075,-0.096030,0.170938,0.723427,-0.091741,0.227759,0.771976,-0.123583,0.207771,0.800647,-0.111719,0.182577,0.788167,-0.097839,A


## Combine Kata + Karakter

In [14]:
kata_df = pd.read_csv('./datasets/extraction-feature-default/40-kata.csv')
kata_df.head()

,filename,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,...,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,label
0,annotated_000008.jpg,0.268382,0.744393,6.803466e-08,0.280761,0.705737,-0.005387,0.332624,0.671905,-0.016733,0.390121,0.661721,-0.028926,0.434973,0.666202,-0.040863,0.342549,0.663610,-0.029849,0.407459,0.637570,-0.050515,0.452237,0.622324,-0.063267,...,0.379611,0.712160,-0.055015,0.450676,0.693179,-0.076478,0.430502,0.699660,-0.062527,0.402310,0.706136,-0.047598,0.393368,0.738127,-0.067984,0.447547,0.719188,-0.075345,0.427779,0.722588,-0.059219,0.401511,0.728171,-0.045366,Aku
1,annotated_000009.jpg,0.299242,0.721207,-2.027723e-09,0.309196,0.684461,-0.002298,0.356788,0.653571,-0.008736,0.410765,0.644649,-0.016566,0.451170,0.647947,-0.024366,0.371834,0.644341,-0.017050,0.431443,0.620406,-0.031288,0.470434,0.605882,-0.040382,...,0.414941,0.689099,-0.037593,0.474138,0.667484,-0.053275,0.455545,0.672951,-0.041676,0.431139,0.680876,-0.028847,0.429697,0.712821,-0.047924,0.474424,0.692056,-0.052460,0.456571,0.694199,-0.038174,0.434866,0.701169,-0.025463,Aku
2,annotated_000010.jpg,0.331698,0.707971,-5.837437e-08,0.337825,0.671227,0.002794,0.376913,0.641802,-0.000845,0.423526,0.634469,-0.007162,0.457788,0.640432,-0.013307,0.400992,0.628433,-0.006874,0.454656,0.606864,-0.016856,0.488799,0.594298,-0.023103,...,0.446112,0.668429,-0.028206,0.490581,0.654379,-0.038300,0.475732,0.661618,-0.028060,0.453976,0.667852,-0.017923,0.459798,0.690260,-0.038764,0.493672,0.677070,-0.038842,0.478665,0.681462,-0.025088,0.459762,0.686913,-0.013507,Aku
3,annotated_000011.jpg,0.353782,0.693124,-7.779809e-08,0.361236,0.657339,0.004973,0.398713,0.630149,0.002905,0.443103,0.623765,-0.001971,0.475117,0.630268,-0.006516,0.419666,0.617920,-0.003185,0.471352,0.597687,-0.010411,0.503044,0.587105,-0.014563,...,0.463501,0.655849,-0.023302,0.505082,0.642756,-0.029734,0.492441,0.649524,-0.019620,0.472496,0.655271,-0.010513,0.477001,0.676810,-0.033037,0.508419,0.664395,-0.030818,0.495135,0.668290,-0.017490,0.477694,0.673256,-0.006726,Aku
4,annotated_000012.jpg,0.377396,0.683487,-2.050306e-07,0.385351,0.645810,0.012018,0.420234,0.619822,0.014677,0.458285,0.613827,0.013094,0.486520,0.619899,0.011651,0.450731,0.606407,0.011326,0.495708,0.591359,0.012575,0.519797,0.584686,0.012652,...,0.495351,0.639933,-0.009315,0.525802,0.631792,-0.005976,0.514033,0.639046,0.001649,0.496737,0.645033,0.005914,0.508515,0.660503,-0.018599,0.533507,0.651758,-0.010427,0.521284,0.656520,0.001844,0.504741,0.662198,0.010487,Aku


In [16]:
df_combined = pd.concat([alphabet_df, kata_df], ignore_index=True)
df_combined.to_csv('./datasets/extraction-feature-default/fix-data.csv', index=False)

df_combined.head()

,filename,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,...,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,label
0,A_0.jpg,0.155182,0.729192,2.719984e-07,0.205288,0.735770,-0.017337,0.275075,0.718441,-0.035706,0.329495,0.709703,-0.055288,0.374989,0.702892,-0.076279,0.293241,0.639296,-0.046337,0.369748,0.612025,-0.084543,0.422023,0.597921,-0.107750,...,0.214076,0.620907,-0.074296,0.272353,0.658624,-0.120460,0.250701,0.692648,-0.112077,0.228995,0.689233,-0.097932,0.166513,0.630084,-0.089888,0.217268,0.657916,-0.120492,0.212754,0.687127,-0.110377,0.198819,0.687841,-0.097879,A
1,A_1.jpg,0.838807,0.719874,-2.164804e-07,0.771837,0.725273,-0.009544,0.705295,0.701828,-0.023455,0.662583,0.677778,-0.039253,0.628528,0.655313,-0.056163,0.697544,0.624706,-0.041077,0.612177,0.596289,-0.075613,0.560344,0.577005,-0.094528,...,0.778687,0.613676,-0.073650,0.714158,0.669719,-0.111475,0.741234,0.699804,-0.090671,0.767731,0.690158,-0.068801,0.825814,0.631450,-0.092105,0.772004,0.671297,-0.121461,0.778908,0.694842,-0.108882,0.794608,0.690194,-0.093175,A
2,A_10.jpg,0.581184,0.745845,-1.059189e-07,0.557238,0.751812,-0.004449,0.536100,0.740906,-0.010580,0.519795,0.737985,-0.017672,0.502753,0.742129,-0.025208,0.539090,0.672356,-0.008672,0.520708,0.655394,-0.017746,0.509604,0.638421,-0.024760,...,0.561821,0.671356,-0.017525,0.537792,0.710742,-0.025926,0.544868,0.730442,-0.019770,0.553987,0.724883,-0.015368,0.572253,0.688464,-0.023423,0.551785,0.715056,-0.030928,0.554675,0.728463,-0.027442,0.562592,0.725510,-0.023784,A
3,A_100.jpg,0.374870,0.759894,4.366655e-08,0.398134,0.758703,-0.007355,0.421602,0.743577,-0.017515,0.442338,0.741927,-0.028168,0.462645,0.746148,-0.039076,0.413038,0.671357,-0.023561,0.437463,0.646744,-0.040337,0.453789,0.627753,-0.050616,...,0.383540,0.671320,-0.034483,0.411728,0.706482,-0.052234,0.408348,0.740305,-0.046356,0.400300,0.747717,-0.039503,0.370315,0.691556,-0.041090,0.393848,0.708879,-0.052244,0.397180,0.733071,-0.047251,0.392820,0.741922,-0.041883,A
4,A_102.jpg,0.143311,0.835336,1.923219e-07,0.214950,0.844010,-0.012342,0.296637,0.814950,-0.027893,0.355519,0.792106,-0.046894,0.400802,0.763687,-0.066535,0.302602,0.724712,-0.037958,0.395459,0.693706,-0.078667,0.456062,0.675371,-0.102087,...,0.215821,0.706110,-0.072580,0.282703,0.770142,-0.123735,0.246608,0.804842,-0.112590,0.217682,0.791075,-0.096030,0.170938,0.723427,-0.091741,0.227759,0.771976,-0.123583,0.207771,0.800647,-0.111719,0.182577,0.788167,-0.097839,A


In [19]:
df_combined['label'].value_counts()

label
Siapa           1089
Dia              562
Maaf             513
Halo             512
Kerja            442
Tolong           413
Aku              384
Bapak            384
Besok            380
Bagaimana        377
Pagi             376
Kemarin          376
Kantor           370
Tidur            363
Jawab            362
Malam            354
Kalian           352
Nanti            347
Siang            346
T                334
H                331
S                330
L                328
F                326
Hari ini         326
B                324
K                319
J                318
Lusa             318
P                317
Lelah            314
W                312
Kamu             312
E                310
Sore             309
Sekarang         307
Kapan            306
X                304
V                298
N                298
Q                297
R                296
C                293
M                291
Z                288
I                287
D                283
A      

# Pre Modelling

In [29]:
df = pd.read_csv('./datasets/extraction-feature-default/fix-data.csv')
df.head()

,filename,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,...,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,label
0,A_0.jpg,0.155182,0.729192,2.719984e-07,0.205288,0.735770,-0.017337,0.275075,0.718441,-0.035706,0.329495,0.709703,-0.055288,0.374989,0.702892,-0.076279,0.293241,0.639296,-0.046337,0.369748,0.612025,-0.084543,0.422023,0.597921,-0.107750,...,0.214076,0.620907,-0.074296,0.272353,0.658624,-0.120460,0.250701,0.692648,-0.112077,0.228995,0.689233,-0.097932,0.166513,0.630084,-0.089888,0.217268,0.657916,-0.120492,0.212754,0.687127,-0.110377,0.198819,0.687841,-0.097879,A
1,A_1.jpg,0.838807,0.719874,-2.164804e-07,0.771837,0.725273,-0.009544,0.705295,0.701828,-0.023455,0.662583,0.677778,-0.039253,0.628528,0.655313,-0.056163,0.697544,0.624706,-0.041077,0.612177,0.596289,-0.075613,0.560344,0.577005,-0.094528,...,0.778687,0.613676,-0.073650,0.714158,0.669719,-0.111475,0.741234,0.699804,-0.090671,0.767731,0.690158,-0.068801,0.825814,0.631450,-0.092105,0.772004,0.671297,-0.121461,0.778908,0.694842,-0.108882,0.794608,0.690194,-0.093175,A
2,A_10.jpg,0.581184,0.745845,-1.059189e-07,0.557238,0.751812,-0.004449,0.536100,0.740906,-0.010580,0.519795,0.737985,-0.017672,0.502753,0.742129,-0.025208,0.539090,0.672356,-0.008672,0.520708,0.655394,-0.017746,0.509604,0.638421,-0.024760,...,0.561821,0.671356,-0.017525,0.537792,0.710742,-0.025926,0.544868,0.730442,-0.019770,0.553987,0.724883,-0.015368,0.572253,0.688464,-0.023423,0.551785,0.715056,-0.030928,0.554675,0.728463,-0.027442,0.562592,0.725510,-0.023784,A
3,A_100.jpg,0.374870,0.759894,4.366655e-08,0.398134,0.758703,-0.007355,0.421602,0.743577,-0.017515,0.442338,0.741927,-0.028168,0.462645,0.746148,-0.039076,0.413038,0.671357,-0.023561,0.437463,0.646744,-0.040337,0.453789,0.627753,-0.050616,...,0.383540,0.671320,-0.034483,0.411728,0.706482,-0.052234,0.408348,0.740305,-0.046356,0.400300,0.747717,-0.039503,0.370315,0.691556,-0.041090,0.393848,0.708879,-0.052244,0.397180,0.733071,-0.047251,0.392820,0.741922,-0.041883,A
4,A_102.jpg,0.143311,0.835336,1.923219e-07,0.214950,0.844010,-0.012342,0.296637,0.814950,-0.027893,0.355519,0.792106,-0.046894,0.400802,0.763687,-0.066535,0.302602,0.724712,-0.037958,0.395459,0.693706,-0.078667,0.456062,0.675371,-0.102087,...,0.215821,0.706110,-0.072580,0.282703,0.770142,-0.123735,0.246608,0.804842,-0.112590,0.217682,0.791075,-0.096030,0.170938,0.723427,-0.091741,0.227759,0.771976,-0.123583,0.207771,0.800647,-0.111719,0.182577,0.788167,-0.097839,A


In [31]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
filename,21029,17139,annotated_000115.jpg,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,21029.0,NaN,NaN,NaN,0.456392,0.168098,0.020867,0.327206,0.460756,0.565225,0.961504
y_0,21029.0,NaN,NaN,NaN,0.647017,0.14905,0.025985,0.553168,0.661717,0.752594,1.058611
z_0,21029.0,NaN,NaN,NaN,0.0,0.0,-0.000001,-0.0,0.0,0.0,0.000002
x_1,21029.0,NaN,NaN,NaN,0.46154,0.150118,-0.00495,0.354698,0.467182,0.547377,0.930616
y_1,21029.0,NaN,NaN,NaN,0.620579,0.149475,0.075829,0.5216,0.637112,0.73109,1.005891
z_1,21029.0,NaN,NaN,NaN,-0.008119,0.013805,-0.11606,-0.011577,-0.004995,-0.000412,0.119151
x_2,21029.0,NaN,NaN,NaN,0.466074,0.135987,-0.017549,0.3814,0.472712,0.539824,0.963333
y_2,21029.0,NaN,NaN,NaN,0.58955,0.149602,0.070293,0.488697,0.606616,0.703065,0.978586
z_2,21029.0,NaN,NaN,NaN,-0.016719,0.021472,-0.18639,-0.023873,-0.010961,-0.004163,0.14765


In [33]:
df.shape

(21029, 65)

In [22]:
X = df.drop(columns=['filename', 'label'])
y = df['label']

display(X.head(), y.head())

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,x_8,...,z_12,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,0.155182,0.729192,2.719984e-07,0.205288,0.735770,-0.017337,0.275075,0.718441,-0.035706,0.329495,0.709703,-0.055288,0.374989,0.702892,-0.076279,0.293241,0.639296,-0.046337,0.369748,0.612025,-0.084543,0.422023,0.597921,-0.107750,0.461456,...,-0.119020,0.214076,0.620907,-0.074296,0.272353,0.658624,-0.120460,0.250701,0.692648,-0.112077,0.228995,0.689233,-0.097932,0.166513,0.630084,-0.089888,0.217268,0.657916,-0.120492,0.212754,0.687127,-0.110377,0.198819,0.687841,-0.097879
1,0.838807,0.719874,-2.164804e-07,0.771837,0.725273,-0.009544,0.705295,0.701828,-0.023455,0.662583,0.677778,-0.039253,0.628528,0.655313,-0.056163,0.697544,0.624706,-0.041077,0.612177,0.596289,-0.075613,0.560344,0.577005,-0.094528,0.519345,...,-0.091366,0.778687,0.613676,-0.073650,0.714158,0.669719,-0.111475,0.741234,0.699804,-0.090671,0.767731,0.690158,-0.068801,0.825814,0.631450,-0.092105,0.772004,0.671297,-0.121461,0.778908,0.694842,-0.108882,0.794608,0.690194,-0.093175
2,0.581184,0.745845,-1.059189e-07,0.557238,0.751812,-0.004449,0.536100,0.740906,-0.010580,0.519795,0.737985,-0.017672,0.502753,0.742129,-0.025208,0.539090,0.672356,-0.008672,0.520708,0.655394,-0.017746,0.509604,0.638421,-0.024760,0.501374,...,-0.023058,0.561821,0.671356,-0.017525,0.537792,0.710742,-0.025926,0.544868,0.730442,-0.019770,0.553987,0.724883,-0.015368,0.572253,0.688464,-0.023423,0.551785,0.715056,-0.030928,0.554675,0.728463,-0.027442,0.562592,0.725510,-0.023784
3,0.374870,0.759894,4.366655e-08,0.398134,0.758703,-0.007355,0.421602,0.743577,-0.017515,0.442338,0.741927,-0.028168,0.462645,0.746148,-0.039076,0.413038,0.671357,-0.023561,0.437463,0.646744,-0.040337,0.453789,0.627753,-0.050616,0.465867,...,-0.047230,0.383540,0.671320,-0.034483,0.411728,0.706482,-0.052234,0.408348,0.740305,-0.046356,0.400300,0.747717,-0.039503,0.370315,0.691556,-0.041090,0.393848,0.708879,-0.052244,0.397180,0.733071,-0.047251,0.392820,0.741922,-0.041883
4,0.143311,0.835336,1.923219e-07,0.214950,0.844010,-0.012342,0.296637,0.814950,-0.027893,0.355519,0.792106,-0.046894,0.400802,0.763687,-0.066535,0.302602,0.724712,-0.037958,0.395459,0.693706,-0.078667,0.456062,0.675371,-0.102087,0.505143,...,-0.119402,0.215821,0.706110,-0.072580,0.282703,0.770142,-0.123735,0.246608,0.804842,-0.112590,0.217682,0.791075,-0.096030,0.170938,0.723427,-0.091741,0.227759,0.771976,-0.123583,0.207771,0.800647,-0.111719,0.182577,0.788167,-0.097839


0    A
1    A
2    A
3    A
4    A
Name: label, dtype: object

## Transform Data

In [25]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

display(X_scaled, y_encoded)

array([[-1.79191179,  0.55133813,  0.63723788, ..., -1.75346966,
         0.63598354, -1.55385967],
       [ 2.27501108,  0.48881885, -0.90667132, ...,  2.2105794 ,
         0.64988857, -1.42389463],
       [ 0.74239976,  0.66306662, -0.55722558, ...,  0.66687564,
         0.85857377,  0.49314218],
       ...,
       [-0.95023254,  0.78702703, -1.28799898, ..., -0.6442551 ,
         1.19216185, -0.39003108],
       [-0.94131954,  0.77505065, -1.10093775, ..., -0.61823988,
         1.21702634, -0.39084381],
       [ 0.69390835,  0.79210944,  0.4529996 , ...,  0.55006843,
         0.47736684, -0.30100494]])

array([ 0,  0,  0, ..., 59, 59, 59])

# Split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

In [42]:
X_train.shape

(16823, 63)

In [37]:
total_class = len(set(y_encoded))
print(total_class)

66


# Modelling

In [43]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(total_class, activation="softmax")) 

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Visualisasi akurasi
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Visualisasi kerugian
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



Epoch 1/10


526/526 [==============================] - 2s 2ms/step - loss: 2.6311 - accuracy: 0.3722 - val_loss: 1.7408 - val_accuracy: 0.5706
Epoch 2/10
526/526 [==============================] - 1s 1ms/step - loss: 1.3569 - accuracy: 0.6467 - val_loss: 1.1268 - val_accuracy: 0.7014
Epoch 3/10
526/526 [==============================] - 1s 1ms/step - loss: 0.9515 - accuracy: 0.7434 - val_loss: 0.8664 - val_accuracy: 0.7694
Epoch 4/10
526/526 [==============================] - 1s 1ms/step - loss: 0.7617 - accuracy: 0.7949 - val_loss: 0.7434 - val_accuracy: 0.7948
Epoch 5/10
526/526 [==============================] - 1s 1ms/step - loss: 0.6449 - accuracy: 0.8222 - val_loss: 0.6430 - val_accuracy: 0.8286
Epoch 6/10
526/526 [==============================] - 1s 1ms/step - loss: 0.5674 - accuracy: 0.8430 - val_loss: 0.5834 - val_accuracy: 0.8388
Epoch 7/10
526/526 [==============================] - 1s 1ms/step - loss: 0.5072 - accuracy: 0.8564 - val_loss: 0.5560 - val_accuracy: 0.8459
Ep